In [1]:
import numpy as np
import pandas as pd 

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [6]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [10]:
x_train

,age,gender,fever,cough,city
75,5,Male,102.0,Mild,Kolkata
85,16,Female,103.0,Mild,Bangalore
6,14,Male,101.0,Strong,Bangalore
12,25,Female,99.0,Strong,Kolkata
38,49,Female,101.0,Mild,Delhi
...,...,...,...,...,...
65,69,Female,102.0,Mild,Bangalore
90,59,Female,99.0,Strong,Delhi
58,23,Male,98.0,Strong,Mumbai
2,42,Male,101.0,Mild,Delhi


# Without ColumnTransformer 

In [11]:
# adding simple imputer to fever col 
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data 
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [13]:
# OrdinalEncoder -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data 
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [15]:
# OneHotEncoder -> gender, city 
ohe = OneHotEncoder(drop='first', sparse = False)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])

# also the test data 
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

x_train_gender_city.shape

D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [18]:
# Extracting age 
x_train_age = x_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

# also the test data 
x_test_age = x_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

x_train_age.shape

(80, 1)

In [19]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)

x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)
x_train_transformed.shape

(80, 7)

# With ColumnTransformer 

In [20]:
from sklearn.compose import ColumnTransformer

In [21]:
transformer  = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [22]:
transformer.fit_transform(x_train)

array([[102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  16.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  25.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          0.    

In [23]:
transformer.fit_transform(x_train).shape

(80, 7)

In [24]:
transformer.transform(x_test).shape

(20, 7)